In [1]:
!nvidia-smi

Mon Aug 22 10:15:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 30%   33C    P8    20W / 320W |   7424MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch

In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import re, magic

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage import zoom
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

#vit
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [3]:
model =timm.create_model('efficientnet_lite0', pretrained=True, num_classes=2)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU6(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU6(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_

In [6]:
trn_dir = glob('../data/train_data/nbi_wl(pacs+testvideo)/*.png')
trn_df = pd.DataFrame(trn_dir, columns=['image_id'])
trn_df['image_id'] = trn_df['image_id'].apply(lambda x: os.path.basename(x))
trn_df['label'] = trn_df['image_id'].apply(lambda x: 1 if 'N' in x else 0)
trn_df
train=trn_df

In [7]:
train

,image_id,label
0,S00145_01_W.png,0
1,P00462_08_W.png,0
2,P00676_05_N.png,1
3,V1003_p3_0018_N.png,1
4,S02606_01_W.png,0
...,...,...
37959,HP_S03850_01_W.png,0
37960,V0564_p1_0063_W.png,0
37961,S01856_01_N.png,1
37962,AD_S04978_03_N.png,1


In [9]:
train.label.unique()

array([0, 1])

In [6]:
train.label.value_counts()

1    21548
0    16416
Name: label, dtype: int64

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [8]:
class ColonDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [9]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize, SmallestMaxSize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            #SmallestMaxSize(CFG['img_size']),
            RandomResizedCrop(CFG['img_size'], CFG['img_size'], p=0.5),
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


def get_inference_transforms():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [10]:
class ColonImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class=2, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=n_class)
        try :
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)

        except :
            
            try : 
                n_features = self.model.fc.in_features
                self.model.fc = nn.Linear(n_features, n_class)
            
            except :
                try:
                    n_features = self.model.head.fc.in_channels
                    self.model.head.fc = nn.Conv2d(n_features,n_class,kernel_size=(1, 1), stride=(1, 1))
            
                except:
                    n_features = self.model.head.in_features
                    self.model.head = nn.Linear(n_features, n_class)
            
    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [12]:
CFG = {
    'fold_num': 5,
    'seed': 25,
    'model_arch': 'efficientnet_lite0',
    'img_size': 384,
    'epochs': 100,
    'train_bs': 10,
    'valid_bs': 10,
    'lr': 1e-5,
    'num_workers': 0,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 1,
    'used_epochs': [5],
    'weights': [1]
}

In [13]:
tst_dir = glob('D:/@image_data/@SSP_crop/*.png')
tst_df = pd.DataFrame(tst_dir, columns=['image_id'])
tst_df['image_id'] = tst_df['image_id'].apply(lambda x: os.path.basename(x))
tst_df['label'] = tst_df['image_id'].apply(lambda x: 1 if 'N' in x else 0)
tst_df
test=tst_df

In [34]:
test

,image_id,label
0,P00019_01_W.png,0
1,P00019_02_W.png,0
2,P00019_03_W.png,0
3,P00019_04_W.png,0
4,P00019_05_N.png,1
...,...,...
1488,P04913_06_W.png,0
1489,P04928_01_W.png,0
1490,P04928_02_N.png,1
1491,P04928_03_N.png,1


In [17]:
test['image_id'] = [x for x in os.listdir(test_dir ) ]
test

,image_id
0,P00019_01_W.png
1,P00019_01_W.txt
2,P00019_02_W.png
3,P00019_02_W.txt
4,P00019_03_W.png
...,...
2982,P04928_03_N.png
2983,P04928_03_N.txt
2984,P04928_04_W.png
2985,P04928_04_W.txt


In [18]:
train_dir = 'D:/@image_data/nbi_wl(pacs+testvideo)'
test_dir ='D:/@image_data/@SSP_crop'
model_dir = '../models_/220930/efficientnet_lite0/pacs+video_{}_fold_0_{}'.format(CFG['model_arch'], CFG['used_epochs'][0])


test = pd.DataFrame()
test['image_id'] = [x for x in os.listdir(test_dir) if x.endswith("png") ]
test_ds = ColonDataset(test, test_dir, transforms=get_valid_transforms(), output_label=False)
tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,)
device = torch.device(CFG['device'])
model = ColonImgClassifier(CFG['model_arch'], train.label.nunique()).to(device)
tst_preds=[]
for i, epoch in enumerate(CFG['used_epochs']):
    model.load_state_dict(torch.load(model_dir) )  ## 모델 위치 수정해서 돌리면 됨!
    with torch.no_grad():
        for _ in range(CFG['tta']):
            tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]
                    
                    

tst_preds = np.mean(tst_preds, axis=0) 
        
        
del model
torch.cuda.empty_cache()
        
        # we'll train fold 0 first


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:48<00:00,  3.11it/s]


In [16]:
test

,image_id,pred,confidence score,label
0,V0040_p1_NC.mp4_0001._W.png,0,0.999897,1
1,V0040_p1_NC.mp4_0089._W.png,0,0.999605,1
2,V0040_p1_NC.mp4_0090._W.png,0,0.998143,1
3,V0040_p1_NC.mp4_0091._W.png,0,0.999920,1
4,V0040_p1_NC.mp4_0092._W.png,0,0.999765,1
...,...,...,...,...
21777,V1824_p5_HP.mp4_1015._W._W.png,0,0.998817,0
21778,V1824_p5_HP.mp4_1016._W._W.png,0,0.999734,0
21779,V1824_p5_HP.mp4_1017._W._W.png,0,0.999559,0
21780,V1824_p5_HP.mp4_1018._W._W.png,0,0.999911,0


In [41]:
def score(df):
    TP=len(df.query("label==1 and pred==1 "))
    TN=len(df.query("label==0 and pred==0 "))
    FP=len(df.query("label==0 and pred==1 "))
    FN=len(df.query("label==1 and pred==0 "))
    Sen=round(TP/(TP+FN),4)
    Spe=round(TN/(TN+FP),4) if TN+FP != 0 else 0
    PPV=round(TP/(TP+FP),4)
    NPV=round(TN/(TN+FN),4) if TN+FN != 0 else 0
    ACC=round((TP+TN)/(TP+TN+FP+FN),4)
    print("TP:",TP)
    print("FP:",FP)
    print("TN:",TN)
    print("FN:",FN)
    print("Sen:",Sen)
    print("Spe:",Spe)
    print("PPV:",PPV)
    print("NPV:",NPV)
    print("ACC:",ACC)